In [0]:
%run ../utils/common

In [0]:

dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_mch3
(
  mch3_id STRING,
  mch3_name STRING,
  mch3_desc STRING
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
 
""")

In [0]:
spark.sql(f"""CREATE OR REPLACE TEMP VIEW temp_d_mch3 AS
SELECT 
a._bic_zc_mchlv3 AS mch3_id,  
a.txtsh AS mch3_name,
a.txtmd AS mch3_desc
FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY _bic_zc_mchlv3 ORDER BY proc_date DESC) rn 
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_zc_mchlv3
        WHERE end_date = '2400-01-01'
    ) a WHERE rn = 1 
""")

In [0]:
spark.sql(f"""
MERGE INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_mch3 AS main_tbl
USING temp_d_mch3 AS temp_tbl
ON main_tbl.mch3_id = temp_tbl.mch3_id

WHEN MATCHED THEN 
    UPDATE SET 
        main_tbl.mch3_name = temp_tbl.mch3_name,
        main_tbl.mch3_desc = temp_tbl.mch3_desc

WHEN NOT MATCHED THEN 
    INSERT (mch3_id, mch3_name, mch3_desc)
    VALUES (temp_tbl.mch3_id, temp_tbl.mch3_name, temp_tbl.mch3_desc)
""")


In [0]:
spark.sql("DROP VIEW IF EXISTS temp_d_mch3")
